In [8]:
# Este es mi código sobre mi proyecto 1 de programación.
# Para esta práctica, se codificara un algoritmo para ordenar las actividades de
# un proyecto con el fin de elaborarlo en el menor tiempo posible. Ahora, las
# actividades satisfacen restricciones dadas por una relación de precedencia,
# la cual resulta ser transitiva.

# Objetivo
# Para esta práctica, se deberá codificar un programa que pueda retornar el tiempo
# mínimo necesario para la elaboración de un proyecto, así como la lista de sus
# actividades críticas, dada una tabla de excel que contenga las actividades del
# proyecto, su descripción, sus actividades precedentes inmediatas y la duración de cada
# una de ellas.
# Para ello, se deben de seguir los siguientes pasos:
# (i) Transformar los archivos de excel en una red que deberán de seguir las reglas
# de construcción:
# (a) Un único nodo no tiene predecesores y es definido como el nodo inicial.
# (b) Un único nodo no tiene sucesores y es definido como el nodo final.
# (c) Cada actividad debe aparecer una unica vez representada como un arco en la red
# y se encuentra contenida en un camino que recorre desde el nodo inicial al nodo final.
# Se puede emplear el uso de actividades ficticias para el cumplimiento de esta regla.
# (d) Hay un camino que pasa sucesivamente a traves de dos actividades si y sólo
# si la primera es un requisito previo de la segunda.
# (e) Los pesos de los arcos estaran definidos como la duración de la actividad.
# Para los arcos ficticios, utilicen un peso nulo.
# (ii) Posterior a esto, deberán determinar las fechas mas próximas y fechas más
# lejanas para cada nodo. Estos métodos serán revisados en las ayudantías.
# (iii) Con lo anterior, el programa deberá generar un reporte con formato “.txt”
# que contenga el tiempo mínimo necesitado para la elaboracion del proyecto, asi
# como las actividades críticas.

import openpyxl
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount("/content/gdrive")
# Aqui se importan las bibliotecas que necesitaremos:
# openpyxl porque vamos a trabajar con archivos de Excel, pandas y numpy para el manejo de
# datos, y drive de Google Colab para montar Google Drive y acceder a los archivos
# almacenados en el

class Archivo:
    """
  La clase Archivo, que tiene un método excel_dictionary() para convierte
  el contenido del archivo de Excel en un diccionario.
    """
    def __init__(self, arch_excel):
        self.arch_excel = arch_excel
        self.dictionary = {}
        """
        Metodo para inicializar la clase Archivo con la ruta del archivo Excel y
        un diccionario vacío.
        """

    def excel_dictionary(self):
        """
      Metodo excel_dictionary, carga el archivo de Excel utilizando openpyxl,
      luego obtiene la hoja activa del archivo.
        """
        self.dictionary = {}
        workbook = openpyxl.load_workbook(self.arch_excel)
        sheet = workbook.active

        encabezado = [sheet.cell(row=1, column=col).value for col in range(1, sheet.max_column + 1)]
        # Extraemos los encabezados de las columnas de la primera fila del archivo de Excel

        for indice_fila in range(2, sheet.max_row + 1):
            actividad = sheet.cell(row=indice_fila, column=1).value
            descripcion = sheet.cell(row=indice_fila, column=2).value
            precedentes = sheet.cell(row=indice_fila, column=3).value
            duracion = sheet.cell(row=indice_fila, column=4).value
        # Aquí iteramos sobre las filas del archivo de Excel (excluyendo la primera
        # fila de encabezados) y extraemos la información de cada fila, en este caso la actividad,
        # descripción, predecesores y duración

            precedentes_lista = [float(p) for p in precedentes.split(",")] if isinstance(precedentes, str) else [precedentes]
            self.dictionary[actividad] =[descripcion, precedentes_lista, duracion]
            # Convertimos la cadena de predecesores en una lista de numeros
            # flotantes si la entrada es una cadena, o deja la entrada tal como
            # esta si no es una cadena. Luego, agrega la información de la actividad
            # al diccionario, utilizando el nombre de la actividad como clave y
            # una lista que contiene la descripcion, la lista de predecesores y la duracion
            # como su valor

        return self.dictionary
        # Devuelve el diccionario creado a partir del archivo de Excel

class Grafica():
  """
  Clase Grafica, que tiene dos atributos: nodos, una lista para almacenar
  los nodos de la gráfica, y arcos, una lista para almacenar los arcos de la gráfica.
  """
  def __init__(self):
    self.nodos = []
    self.arcos = []

  def len_arcos(self):
    """
    Metodo que define la longitud de los arcos
    """
    return len(self.arcos)
    # Devuelve la longitud de los arcos

  def len_nodos(self):
    """
    Metodo que define la longitud de los nodos
    """
    return len(self.nodos)
    # Devuelve la longitud de los nodos

  def antecesores(self,nodo):
    """
    Metodo que toma un nodo como entrada y devuelve una lista de los antecesores
    del nodo en la gráfica
    """
    if nodo not in self.nodos:
      print(nodo," no pertenece al conjunto de nodos")
      # Si el nodo no está en la lista de nodos, imprime un mensaje de error
      return []
      # devuelve una lista vacia
    else:
      Ant = []
      for x in self.arcos:
        if nodo == x[-1]:
          Ant.append(x[0])
      return Ant

  def sucesores(self,nodo):
    """
    Metodo que toma un nodo como entrada y devuelve una lista de los sucesores del nodo
    en la gráfica
    """
    if nodo not in self.nodos:
      print(nodo," no pertenece al conjunto de nodos")
      # Si el nodo no está en la lista de nodos, imprime un mensaje de error
      return []
      # Devuelve una lista vacía
    else:
      Ant = []
      for x in self.arcos:
        if nodo == x[0]:
          Ant.append(x[-1])
      return Ant

  def add_vertex(self, nodo):
    """
    Metodo encargado de agregar nodos
    """
    self.nodos.append(nodo)

  def add_arc(self, arco):
    """
    Metodo encargado de agregar arcos
    """
    if type(arco) == tuple and len(arco) == 2:
      if arco[0] in self.nodos and arco[1] in self.nodos:
        self.arcos.append(arco)
      else:
        print("Las entradas no pertenecen a los nodos.")
    else:
      print("El arco no es valido.")
    #  Verifica si el arco es válido (un par de nodos) y si los nodos del arco
    # están presentes en la lista de nodos antes de agregarlos. Si no, imprime
    # un mensaje de error.

class MiRed(Grafica):
  """
  Clase MiRed que hereda de la clase Gráfica y en la cual van a estar contenidos
  los metodos principales para la manipulacion de graficas
  """

  def ordenar_actividades(self, dictionary):
        """
    Este método ordena las actividades utilizando un algoritmo de búsqueda en profundidad
        """
        visitados = set()  # Inicializa el conjunto de nodos visitados
        orden = [] # Lista para almacenar las actividades ordenadas

        def dfs(actividad):
            """
          Función de búsqueda en profundidad (DFS) para recorrer las actividades
          y sus predecesores.
            """
            if actividad not in visitados:
                visitados.add(actividad)

                if actividad in dictionary and dictionary[actividad][1]:
                  # Verifica si la actividad tiene predecesores
                    predecesores = dictionary[actividad][1]
                    for predecesor in predecesores:
                      # Llama recursivamente al método para los predecesores
                        # Agrega la actividad al final de la lista
                        dfs(predecesor)
                orden.append(actividad)

        # Itera sobre todas las actividades del diccionario
        for actividad in dictionary:
            # Llama al método de búsqueda para cada actividad
            dfs(actividad)

        return orden[::1]
  def lista_adyacente(self, diccionario):
        """
        Este método crea una lista de adyacencia a partir del diccionario de actividades.
        """
        lista_adyacencia = {}
        # Declaracion del diccionario
        for clave, valor in diccionario.items():
            nodo = clave
            vecinos = valor[1]
            if nodo not in lista_adyacencia:
                lista_adyacencia[nodo] = []
            for vecino in vecinos:
                if vecino is not None:
                    lista_adyacencia[nodo].append(vecino)
        return lista_adyacencia
        # Devuelve el diccionario

  def det_subsecuencia(self, dic):
    """
    Este método toma un diccionario dic como entrada e inicializa un diccionario
    vacío llamado sub.
    """
    sub = {}
    for a, ps in dic.items():
      sub[a] = []
      for p in ps:
        sub[a].append(p)
    return sub
    # Itera sobre cada clave en el diccionario y devuelve el diccionario
    # sub que contiene las subsecuencias de predecesores para cada actividad

  def crear_arcos(self, dic, sub):
    """
    Este método crea los arcos de la red a partir del diccionario de actividades y el diccionario de subsecuencias.
    Inicializa varios atributos de la clase MiRed: arcs, nodes, done y arrows.
    """
    self.arcs = {}
    self.nodes = [0, 1]
    # Primer nodo y último nodo
    self.done = [{}, {}]
    ind = 2
    self.arrows = []

    for k, v in dic.items():
      # Itera sobre cada clave en el diccionario, donde la clave k representa una
      # actividad y el valor v es una lista de predecesores
        if not v:
          # Para cada actividad, verifica si tiene predecesores (v). Si no tiene,
          #crea un nuevo nodo y un arco desde el nodo 0 al nuevo nodo
            self.nodes.append(ind)
            self.arrows.append((0, ind))
            self.done.append({k})
            self.arcs[k] = (0, ind)
            ind += 1
        elif v not in self.done:
          # Si tiene predecesores, comprueba si ya estan registrados en done.
          # Si no lo estan, crea un nuevo nodo y arcos desde los predecesores
          # registrados hasta el nuevo nodo.
            for element in v:
                if element in self.done:
                    self.arrows.append((self.done.index(element), ind))
            self.done.append(v)
            self.nodes.append(ind)
            if not sub[k]:
                self.arrows.append((self.done.index(v), ind))
                self.arcs[k] = (self.done.index(v), 1)
            else:
                self.nodes.append(ind)
                self.arrows.append((self.done.index(v), ind))
                self.done.append({k})
                self.arcs[k] = (self.done.index(v), ind)
                ind += 1
        elif not sub[k]:
            self.arrows.append((self.done.index(v), ind))
            self.arcs[k] = (self.done.index(v), ind)
        else:
            self.nodes.append(ind)
            self.arrows.append((self.done.index(v), ind))
            self.done.append({k})
            self.arcs[k] = (self.done.index(v), ind)
            ind += 1

    return self.done, self.arrows, self.arcs, self.nodes
    # Retorna los atributos done, arrows, arcs, y nodes que representan los nodos,
    # arcos y estado de procesamiento de la red

class ActividadesCriticas:
    """
  La clase ActividadesCriticas se encarga de realizar varios cálculos importantes
  para el análisis de actividades en un proyecto.
    """
    def __init__(self):
        """
      Metodo __init__,que inicializa cuatro atributos: fmp, fml, holgura_actividades,
      actividades_criticas.
        """
        self.fmp = {}
        # Un diccionario que contendrá las fechas mas proximas para cada nodo en
        # el grafo de actividades
        self.fml = {}
        # Un diccionario que contendrá las fechas mas lejanas para cada nodo en
        # el grafo de actividades
        self.holgura_actividades = {}
        self.actividades_criticas = []

    def actividades_a_diccionario(lista_actividades):
      """
      Este metodo convierte una lista de actividades en un diccionario, donde las
      claves son el nombre de la actividad y los valores son los detalles de la actividad.
      """
      diccionario = {}
      for actividad in lista_actividades:
        diccionario[actividad['actividad']] = actividad['arco']
      return diccionario

    def fechas_mas_proximas(self, lista_actividades):
        """
      Calcula las fechas mas proximas para cada nodo en el grafo de actividades.
      Utiliza la tecnica de cálculo hacia adelante para determinar cuándo se
      completara cada actividad.
        """
        for index, actividad in enumerate(lista_actividades):
            nodo_inicial = actividad['arco'][0]
            if nodo_inicial == 1:
                self.fmp[nodo_inicial] = 0
            else:
                duracion = actividad['arco'][1]
                calculo = self.fmp[nodo_inicial - 1] + duracion
                self.fmp[nodo_inicial] = max(calculo, self.fmp.get(nodo_inicial, 0))

    def fechas_mas_lejanas(self, lista_actividades):
        """
       Calcula las fechas mas lejanas para cada nodo en el grafo de actividades.
       Utiliza la técnica de cálculo hacia atrás para determinar cuál es la fecha
       máxima en la que se puede completar cada actividad
        """
        ultimo_nodo = lista_actividades[-1]['arco'][1]
        self.fml[ultimo_nodo] = self.fmp[ultimo_nodo]
        for index in range(len(lista_actividades) - 1, 0, -1):
            actividad = lista_actividades[index]
            nodo_final = actividad['arco'][1]
            duracion = actividad['arco'][1]
            calculo = self.fml[nodo_final + 1] - duracion
            self.fml[nodo_final] = max(calculo, self.fml.get(nodo_final, 0))

    def holgura_actividades(self, arcos):
      """
      Calcula la holgura de cada actividad en el grafo. La holgura de una actividad
      es la cantidad de tiempo adicional que puede demorar sin afectar el tiempo
      total de finalización del proyecto.
      """
      for (nodo_inicial, nodo_final), (actividad, duracion) in arcos.items():
        self.holgura_actividades[actividad] = self.fml[nodo_final] - self.fmp[nodo_inicial] - duracion
      return self.holgura_actividades

    def actividad_critica(self, arcos):
        """
      Identifica las actividades críticas en el proyecto. Una actividad es crítica
      si su holgura es cero, lo que significa que cualquier retraso en esa actividad
      retrasara el proyecto en su totalidad.
        """
        for (nodo_inicial, nodo_final), (actividad, duracion) in arcos.items():
            if self.holgura_actividades[actividad] == 0 and actividad not in self.actividad_critica:
                self.actividad_critica.append(actividad)
        return self.actividad_critica

class ReporteFinal_txt:
  """
  Metodo que genera un informe en formato de texto que contiene información sobre
  el tiempo necesario para la elaboración del proyecto y las actividades críticas
  junto con sus descripciones.
  """

  def __init__(self, archivo_final, diccionario):
    """
    Método constructor de la clase. Recibe dos parámetros: archivo_final y diccionario.
    """
    self.reporte = archivo_final
    # Almacena la ruta del archivo donde se va a guardar el informe
    self.dictionary = diccionario
    #  Almacena el diccionario que contiene la información necesaria para generar el informe

  def reporte_txt(self, tiempo_minimo, actividad_critica, ordenar_actividades):
    """
    Este metodo genera el informe en formato de texto. Recibe tres parametros:
    tiempo_minimo, actividad_critica y ordenar_actividades.
    """
    actividades_ordenadas = list(sorted(actividad_critica))
    # Utilizamos la funcion sorted de phyton para ordenar las actividades de forma
    # ascendente
    with open(self.reporte, "w") as archivo:
      # Abre el archivo especificado en modo de escritura ("w") utilizando el gestor de contexto with open.
      archivo.write(f"El tiempo necesario para la elaboracion del proyecto {int(tiempo_minimo)}\n")
      archivo.write("Actividades criticas:\n")
      for actividad in ordenar_actividades:
        archivo.write(f"{actividad}: {self.dictionary[actividad][0]}\n")


def main():
    """
  Funcion principal del codigo que realiza acciones importantes
    """

    # Crear objeto de la clase Archivo y obtener el diccionario
    arch_excel = "/content/gdrive/MyDrive/Classroom/Creacion_de_la_carrera_de_mat_ap.xlsx"
    archivo_objeto = Archivo(arch_excel)
    dictionary = archivo_objeto.excel_dictionary()
    print("Lista de actividades:")
    print(list(range(1, len(dictionary) + 1)))

    # Crear objeto de la clase MiRed y obtener la lista de actividades ordenadas
    red_objeto = MiRed()
    actividades_ordenadas = red_objeto.ordenar_actividades(dictionary)
    # Filtrar los valores None y convertir a números enteros
    actividades_ordenadas = [int(actividad) for actividad in actividades_ordenadas if actividad is not None]
    # Imprimir actividades ordenadas
    print("Lista de actividades ordenadas por precedencia:")
    print(actividades_ordenadas)

    # Crear objeto de la clase MiRed y obtener la lista de adyacencia
    red_objeto = MiRed()
    lista_adyacencia = red_objeto.lista_adyacente(dictionary)
    sub = red_objeto.det_subsecuencia(dictionary)

    # Imprimir lista de adyacencia en una sola línea
    print("Lista de adyacencia:")
    print("[", end="")
    print(", ".join(f"Nodo {nodo}: Vecinos {vecinos}" for nodo, vecinos in lista_adyacencia.items()), end="")
    print("]")

    done, arrows, arcs, nodes = red_objeto.crear_arcos(dictionary, sub)

    # Imprimir la lista de arcos en el formato {actividad: (nodo inicial, nodo final)}
    arcos_lista = [{"actividad": actividad, "arco": arco} for actividad, arco in arcs.items()]
    print("Arcos:")
    print(arcos_lista)

    # Imprimir la lista de nodos
    print("Nodos:")
    print(nodes)

main()




# No logré terminar el algoritmo, una disculpa

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Lista de actividades:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
Lista de actividades ordenadas por precedencia:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 16, 17, 19, 20, 21]
Lista de adyacencia:
[Nodo 1: Vecinos [], Nodo 2: Vecinos [1], Nodo 3: Vecinos [2], Nodo 4: Vecinos [3], Nodo 5: Vecinos [4], Nodo 6: Vecinos [4], Nodo 7: Vecinos [4], Nodo 8: Vecinos [4], Nodo 9: Vecinos [4], Nodo 10: Vecinos [5], Nodo 11: Vecinos [6], Nodo 12: Vecinos [7], Nodo 13: Vecinos [8], Nodo 14: Vecinos [11], Nodo 15: Vecinos [10.0, 14.0], Nodo 16: Vecinos [12.0, 13.0, 18.0], Nodo 17: Vecinos [9.0, 16.0], Nodo 18: Vecinos [15], Nodo 19: Vecinos [18], Nodo 20: Vecinos [17.0, 19.0], Nodo 21: Vecinos [20]]
Arcos:
[{'actividad': 1, 'arco': (2, 2)}, {'actividad': 2, 'arco': (4, 3)}, {'actividad': 3, 'arco': (6, 4)}, {'actividad': 4, '